In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path="../../models/hf"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of te

In [3]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, max_new_tokens=200)

In [4]:
pipe("今日はいい")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '今日はいい。\n\n2010年1月1日、NHKの「NHKニュース・報道特番」で、NHKの「NHKニュース・報道特番」で、NHKの「NHKニュース・報道特番」で、NHKの「NHKニュース・報道特番」で、NHKニュース・報道特番で放送された。\n\n2011年1月1日、NHKニュース・報道特番で放送された。\n\n2012年1月1日、NHKニュース・報道特番で放送された。\n\n2013年1月1日、NHKニュース・報道特番で放送された。\n\n2014年1月1日、NHKニュース・報道特番で'}]

In [5]:
pipe("富士山は")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '富士山は、富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓の富士山麓'}]

In [6]:
pipe("i have a")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'i have a slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly slightly'}]